In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set(context = "notebook",  style='whitegrid')

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
#lendo os dados apenas as colunas id e titulo para evitar erro
dados1 = pd.read_csv('projetos-de-pesquisa.csv', sep = ";", quotechar = '"', usecols = ["id_projeto_pesquisa", "titulo"])
columns = ["id_projeto_pesquisa", 'codigo_projeto', "palavras_chave","ano","data_inicio","data_fim","situacao","tipo_projeto",
           "unidade","coordenador","grupo_pesquisa","linha_pesquisa","area_conhecimento_cnpq"]
#lendo as colunas restantes
dados2 = pd.read_csv('projetos-de-pesquisa.csv', sep = ";", quotechar = '"', usecols = columns)
#dando merge nos dois
dados = dados1.merge(dados2, on = "id_projeto_pesquisa").drop([1010,790, 1011], axis = 0)
#convertendo as colunas para datetime
dados.ano = pd.to_datetime(dados.ano)
dados.data_inicio = pd.to_datetime(dados.data_inicio)
dados.data_fim = pd.to_datetime(dados.data_fim)
dados.head()

FileNotFoundError: ignored

In [2]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (12, 6))
dados2 = dados[['ano', 'data_inicio']].copy()
dados2.data_inicio = dados2.data_inicio.apply(lambda x: pd.to_datetime(x.year, format = "%Y")) 
temp = dados2.data_inicio.value_counts()[0:-1]
temp.plot(marker = "o", ax = ax)
ax.set_ylim(0, 1600)
ax.set_xlim("1999", "2019")
ax.set_title("Quantidade de Projetos de Pesquisa por Ano")
ax.set_xlabel("Ano")
ax.set_ylabel("Quantidade de Projetos")

NameError: ignored

In [0]:
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (12, 6))
numero_projetos = temp.resample(rule = "A").first("first")

#cria o modelo
model = sm.tsa.Holt(numero_projetos)
#treina o modelo
predict = model.fit(smoothing_level = 1, smoothing_slope = 0.2)

#faz previsão e plota
numero_projetos.plot(ax= ax, marker = "o", label = "Real")
predict.predict(start = "2000", end = "2022").plot(ax = ax, marker = "o", label = "Previsão")

ax.legend()
ax.set_xlabel("Ano")
ax.set_ylabel("Quantidade de Projetos")
ax.set_title("Quantidade de Projetos por Ano")

#calcula o erro médio
from sklearn.metrics import mean_squared_error
erro = mean_squared_error(numero_projetos.values, predict.predict(start = "2000", end = "2018").values)

ax.text("2000", 1490, "MSE: " + str(round(erro, 1)))
ax.text("2000", 1560, "AIC: " + str(round(predict.aic, 1)))

In [0]:
area_ano = dados[['data_inicio', 'area_conhecimento_cnpq']].copy()
area_ano.data_inicio = dados2.data_inicio.apply(lambda x: pd.to_datetime(x.year, format = "%Y"))
area_ano = area_ano[area_ano.data_inicio < "2019"]
cores = ["#f03a0f", "#2213f0", "#150937", "#5a83bf", "#9389b1", "#038519", "#aac182", "#545021", "#f0cb77", "#d3c32c", 
         "#cc1627", "#ca03a4", "#774363", "#c64666", "#a13bcb"]
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (14, 8))
#pega as 15 maiores áreas
areas_15maiores = area_ano.area_conhecimento_cnpq.value_counts()[0:15].index
#cria um gráfico para cada uma das 15 maiores áreas de pesquisa
for i in range(15):
    area_ano[area_ano.area_conhecimento_cnpq == areas_15maiores[i]].groupby('data_inicio').count().plot(ax = ax, color = cores[i],
                                                                    marker = "o", label = areas_15maiores[i], markersize = 4)
ax.legend(ncol =2, frameon = True, loc = "upper right", labels = areas_15maiores)  
sns.despine(left = True)
ax.set_title("15 Maiores Áreas de Pesquisa")
ax.set_xlabel("Ano")
ax.set_ylabel("Quantidade de Projetos")

In [0]:
unidade = dados[dados.unidade.isin(dados.unidade.value_counts()[0:10].index)]
fig, ax = plt.subplots(figsize = (12, 7))
unidade = pd.DataFrame(unidade.unidade.value_counts())
unidade.columns = ["Quantidade"]
unidade["unidade"] = unidade.index
sns.barplot(x = "unidade", y = "Quantidade", hue = "unidade", data = unidade)
ax.set_xticklabels("")
ax.legend(loc = "best")

#mudar a largura das barras
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

change_width(ax, .37)
#ajeita a legenda
ax.legend(ncol =1, frameon = True, loc = "upper right")
ax.set_xlabel("")
ax.set_ylabel("Quantidade de Projetos")
sns.despine(left = True, right = True)
ax.set_title("10 Maiores Departamentos de Pesquisa")

In [0]:
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
regex = r"[.|,|@|#|!|$|%|*|&|-|)|(|:|/]+"
stopwords = nltk.corpus.stopwords.words('portuguese')
def tokenization(word):
    word = word.lower()
    word = re.sub(regex, "", word)
    word = [palavra for palavra in word.split() if palavra not in stopwords]
    return " ".join(word)
bag = TfidfVectorizer(stop_words= None)
X = bag.fit_transform(dados['titulo'].apply(tokenization))
model = LatentDirichletAllocation(n_components = 3)
model.fit(X)
def print_topics(model, count_vectorizer, n_top_words):
    words = bag.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
print_topics(model, bag, 10)        